In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder
import scipy.sparse

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Indonesia_Hoax_Dataset.xlsx to Indonesia_Hoax_Dataset.xlsx


In [3]:
df = pd.read_excel('Indonesia_Hoax_Dataset.xlsx')

In [4]:
df

,index,title,raw timestamp,original,tags,author,url,cleaned,label,timestamp,cleaned token length,summarized
0,0,Ma'ruf Amin akan Saksikan Lagi Timnas Indonesi...,"Sabtu, 1 Januari 2022 17:14 WIB","TEMPO.CO, Jakarta - Wakil Presiden Ma'ruf Amin...",Ma'ruf Amin;Piala AFF 2020;Indonesia vs Thaila...,Reporter Egi Adyatama Editor Aditya Budiman,https://nasional.tempo.co/read/1545504/maruf-a...,Ma'ruf Amin akan Saksikan Lagi Timnas Indonesi...,0,2022-01-01 17:14:00,41,Timnas Indonesia Vs Thailand akan Tampil Lagi ...
1,1,Menag Yaqut Canangkan 2022 Sebagai Tahun Toler...,"Sabtu, 1 Januari 2022 15:05 WIB","TEMPO.CO, Jakarta - Menteri Agama Yaqut Cholil...",Menag;Yaqut Cholil Qoumas;Gus Yaqut;Toleransi;...,Reporter Egi Adyatama Editor Aditya Budiman,https://nasional.tempo.co/read/1545477/menag-y...,Menag Yaqut Canangkan 2022 Sebagai Tahun Toler...,0,2022-01-01 15:05:00,209,Pemerintah mencanangkan 2022 sebagai Tahun Tol...
2,2,Jokowi Ajak Masyarakat Hadapi 2022 dengan Sema...,"Sabtu, 1 Januari 2022 12:05 WIB","TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",Jokowi;2022;Pandemi Covid-19;Resesi,Reporter Antara Editor Eko Ari Wibowo,https://nasional.tempo.co/read/1545437/jokowi-...,Jokowi Ajak Masyarakat Hadapi 2022 dengan Sema...,0,2022-01-01 12:05:00,40,"@jokowi.net, melaporkan bahwa dia mengajak mas..."
3,3,"Top Nasional: Strategi Hadapi Omicron, Lemhana...","Sabtu, 1 Januari 2022 07:28 WIB","TEMPO.CO, Jakarta - Berita yang banyak menarik...",Omicron;Lemhanas;Kemenkes;WHO;Agus Widjojo,Reporter Tempo.co Editor Eko Ari Wibowo,https://nasional.tempo.co/read/1545377/top-nas...,"Top Nasional: Strategi Hadapi Omicron, Lemhana...",0,2022-01-01 07:28:00,104,Pemerintah Lemhanas Usul Bentuk Kementerian Ba...
4,4,"Mulai Tahun Ini, Menteri Tjahjo Kumolo Minta P...","Sabtu, 1 Januari 2022 07:02 WIB","TEMPO.CO, Jakarta - Menteri Pendayagunaan Apar...",Tjahjo Kumolo;Menpan RB;ASN;PNS;protokol keseh...,Reporter Friski Riana Editor Syailendra Persada,https://nasional.tempo.co/read/1545310/mulai-t...,"Mulai Tahun Ini, Menteri Tjahjo Kumolo Minta P...",0,2022-01-01 07:02:00,85,Menteri Tjahjo Kumolo Minta PNS Apel Pagi Tiap...
...,...,...,...,...,...,...,...,...,...,...,...,...
31327,31327,(HOAX) Foto Raja Salman Mengangkat Tandu Jenaz...,2015-09-15 00:00:00,Sumber : media sosial\nNarasi : Raja Salman da...,Fitnah;Hasut;Hoax,Bentang Febrylian,https://turnbackhoax.id/2015/09/15/hoax-foto-r...,Raja Salman dari arab saudi membawa orang-oran...,1,2015-09-15 00:00:00,36,Raja Salman dari arab saudi membawa orang-oran...
31328,31328,"(HOAX) Jokowi dituduh bodoh, salah cara dalam ...",2015-09-12 00:00:00,"Sumber : media sosial\nNarasi : hehe, selalu b...",Fitnah;Hasut;Hoax,Bentang Febrylian,https://turnbackhoax.id/2015/09/12/jokowi-ditu...,"hehe, selalu bisa tersenyum melihat tingkah la...",1,2015-09-12 00:00:00,32,Jokowi selalu bisa tersenyum melihat tingkah l...
31329,31329,(FITNAH) Jokowi dituduh memasukkan ribuan tena...,2015-09-08 00:00:00,Sumber : Media Sosial\nNarasi : Pak Jokowi itu...,Fitnah;Hasut;Hoax,Bentang Febrylian,https://turnbackhoax.id/2015/09/08/jokowi-ditu...,Pak Jokowi itu menjadi walikota periode pertam...,1,2015-09-08 00:00:00,143,Jokowi bersalah karena lebih mementingkan chin...
31330,31330,(HOAX) Mata Uang Rupiah Terpuruk,2015-09-08 00:00:00,"Sumber : media sosial\nNarasi : Hari ini, Rabu...",Fitnah;Hasut;Hoax,Bentang Febrylian,https://turnbackhoax.id/2015/09/08/mata-uang-r...,"Hari ini, Rabu (23/09/2015), nilai tukar rupia...",1,2015-09-08 00:00:00,31,Nilai tukar rupiah terpuruk hingga sempat mele...


In [5]:
# Mapping label menjadi angka
label_mapping = {'Real': 1, 'Fake': 0}
df['label'] = df['label'].map(label_mapping)

# Pastikan mapping berhasil
print(df['label'].value_counts())

Series([], Name: count, dtype: int64)


In [7]:
X = df['title']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
label_encoder = LabelEncoder()

# Fit the encoder to your training labels
label_encoder.fit(y_train)


LabelEncoder()

In [9]:
max_vocab_size = 10000
sequence_length = 100

In [10]:
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_size,  # Jumlah kata maksimal
    output_mode='int',          # Mengubah teks menjadi indeks
    output_sequence_length=sequence_length  # Panjang sequence yang seragam
)

# Adapt vectorizer ke data teks
vectorizer.adapt(X_train)

In [11]:
# Mengubah teks menjadi numerik
X_train_vectorized= vectorizer(X_train)
X_test_vectorized = vectorizer(X_test)


In [12]:
# Transform the training and testing labels
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [13]:
# Model Sequential dengan lapisan embedding dan LSTM
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=sequence_length),  # Embedding layer
    LSTM(64, return_sequences=True),  # LSTM Layer
    LSTM(32),                         # LSTM Layer kedua
    Dense(1, activation='sigmoid')    # Output layer untuk klasifikasi biner
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [15]:
# Melatih model
history = model.fit(
    X_train_vectorized.numpy(),
    y_train_encoded,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)
if scipy.sparse.issparse(X_test_vectorized):
  X_test_evaluated = X_test_vectorized.toarray()
else:
  X_test_evaluated = X_test_vectorized
# Check the shape of your input data and the input shape of the model
print("Shape of X_test_evaluated:", X_test_evaluated.shape)
print("Input shape of the model:", model.input_shape)
# Convert X_test_vectorized and y_test to TensorFlow tensors
X_test_tensor = tf.convert_to_tensor(X_test_vectorized.numpy())
y_test_tensor = tf.convert_to_tensor(y_test_encoded)
# Evaluasi model
loss, accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print(f"Model Accuracy: {accuracy*100:.2f}%")

Epoch 1/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 89s 132ms/step - accuracy: 1.0000 - loss: 0.0233 - val_accuracy: 1.0000 - val_loss: 9.7523e-05
Epoch 2/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 80s 127ms/step - accuracy: 1.0000 - loss: 7.7347e-05 - val_accuracy: 1.0000 - val_loss: 3.9728e-05
Epoch 3/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 82s 128ms/step - accuracy: 1.0000 - loss: 3.3794e-05 - val_accuracy: 1.0000 - val_loss: 2.0934e-05
Epoch 4/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 82s 129ms/step - accuracy: 1.0000 - loss: 1.8398e-05 - val_accuracy: 1.0000 - val_loss: 1.2474e-05
Epoch 5/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 83s 130ms/step - accuracy: 1.0000 - loss: 1.1164e-05 - val_accuracy: 1.0000 - val_loss: 7.9679e-06
Epoch 6/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 81s 128ms/step - accuracy: 1.0000 - loss: 7.2116e-06 - val_accuracy: 1.0000 - val_loss: 5.3133e-06
Epoch 7/10
627/627 ━━━━━━━━━━━━━━━━━━━━ 81s 127ms/step - accuracy: 1.0000 - loss: 4.8441e-06 - val_accuracy: 1.0000 - val_loss: 3.6441e-06
Epoch 8/10
627/627 ━━━━━━━━━━━━